<h1> 1.<b>前置作業</b> </h1>

<h4> 注意：要先載ckiptagger </h4>
<ul>
  <li> <a href="https://github.com/ckiplab/ckiptagger">Github官方文檔</a></li>
  <li> <a href="https://medium.com/@br19920702/%E4%B8%AD%E7%A0%94%E9%99%A2%E9%96%8B%E6%BA%90nlp%E5%A5%97%E4%BB%B6-ckiptagger-%E7%B9%81%E4%B8%AD%E4%B8%8D%E7%B5%90%E5%B7%B4-7822fc4efbf">中文安裝教學</a></li>
</ul>

In [5]:
import numpy as np
import pandas as pd
import os
import math
import copy

from ckiptagger import WS, POS, NER
ws = WS("./data")

all_small = pd.read_csv('./士貴/專案資料.csv',engine='python',encoding='utf-8',header=None,names=['日期','標題','內容'])


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



<h1>2.<b>排除遺失值&選擇資料</b></h1>

<p style="color:#009933"> >>優化點1：該用甚麼字篩出符合台積電的文章  </p>

In [6]:
all_small = all_small.dropna()
GG = all_small[(all_small['內容'].str.contains('台積|面板|半導體|台G|GG|蘋概股|晶圓|奈米|製程|貿易戰|中美|大盤|美股|2330|川普')==True) & (all_small['標題'].str.contains('[公告]')==False)]

<h1>3.<b>確認迴圈時間</b></h1>

In [7]:
Date = list(set(GG['日期'].value_counts().index)) #抓出時間

In [8]:
Date.sort()       #讓資料照時間排
Date = Date[1:]   #確保資料為5/29-11/6

<h1>4.<b>實作tf</b></h1>

<p style="color:#009933">  >>優化點2：斷詞能否更準 (到最後他會存成很多csv，去預設資料夾看他切出的詞與分數，是還不錯但仍不好) </p>
<p style="color:#009933">  >>優化點3：tf-idf算法是否仍可改良？ 會不會導致重要的字反而分數較低？例如「開噴」，若每筆文章都提到，代表明天股價應該會下降，但idf會認為他在不同文章出現次數過高，反而使分數下降 </p>
<p style="color:#009933">  >>優化點4：我是以"天"為單位，把所有文章總和起來，給每天一個各詞的tf-idf值，所以和tf-idf原先的目的(找出能夠代表這篇文章的重要詞彙)並不相同 </p>

In [14]:
from ckiptagger import WS, POS, NER
import time

ws = WS("./data")

for date in Date[0:2]:
    tStart = time.time()#計時開始
    print('【'+date+'】')
    
    #第一部分： 【斷詞+計算當天詞集+計算當天出現詞數】
    ### 詞數 v.s 詞集:  詞數可以重複，詞集不行  (例如 1th、3th文章都有 "台積" ， 那詞數會是 2個 、詞集會是 1個
    ws_results = ws(GG[GG['日期']==date]['內容'],sentence_segmentation=False)
    word_list = [] #出現詞數
    word_set = []  #當天詞集
    for article_nth in ws_results:
        for word in article_nth :    
            word_list.append(word)
    word_set = list(set(word_list))
    print('>> 共有 ' + str(len(ws_results)) + '文章'+ ' / ' + str(len(word_set)) + '詞' + ' / ' + str(len(word_list)) + '詞數')

    #第二部ˋ分： 【創建空的Dataframe，欄位是當天有幾篇文章，列是當天的詞集數量】
    zero_data = np.zeros(shape=(len(word_set),len(ws_results)))
    globals()['columns'] = []
    for col in range(len(ws_results)):
        columns.append(str(col+1)+'th')
    total = pd.DataFrame(zero_data,index=word_set,columns=columns)
    check_word = list(total.index)
    
    ##算次數
    for total_col in range(len(total.columns)):
        for text_pos in ws_results[total_col]:
            total.iloc[check_word.index(text_pos),total_col] +=1
    print('>> 次數Dataframe完成')
    
    #深複製tf - idf
    tf = copy.deepcopy(total)
    idf = copy.deepcopy(total)
    
    #tf矩陣計算
    for tf_col in range(len(tf.columns)):
        col_sum = tf.sum(axis=0)[tf_col]
        for tf_row in range(len(tf)):
            if (tf.iloc[tf_row,tf_col] != 0):
                tf.iloc[tf_row,tf_col] = round(tf.iloc[tf_row,tf_col]/col_sum,6)
    print('>> tf矩陣完成')
    
    #idf矩陣成形
    for idf_col in range(len(idf.columns)):
        for idf_text_rol in idf.iloc[np.where(idf.iloc[:,idf_col]!=0)[0],:].index:
            idf.iloc[np.where(idf.index==idf_text_rol)[0],idf_col]=(math.log(len(ws_results)/len(np.where(idf.loc[idf_text_rol]!=0)[0])))
    print('>> idf矩陣完成')
    
    #tf idf互乘
    tfidf = pd.DataFrame(np.array(tf)*np.array(idf),columns=columns,index=word_set)
    print('>> tf-idf矩陣完成')
    
    #給出每天的 詞-tfidf分數 資料表
    tfidf_text = pd.DataFrame(tfidf.sum(axis=1),columns=['分數']) #列總和
    word = copy.deepcopy(tfidf_text.sort_values('分數',ascending=False).index)
    score = copy.deepcopy(tfidf_text.sort_values('分數',ascending=False)['分數'])
    pd.DataFrame({'詞':word,'分數':score}).to_csv(date+'.csv',encoding='utf_8_sig',index=False)
    tEnd = time.time()#計時結束
    print('>>>>>> 結束並存檔 (共' + str(round(tEnd-tStart,2)) + '秒)')

【2019-05-29】
>> 共有 7文章 / 259詞 / 403詞數
>> 次數Dataframe完成
>> tf矩陣完成
>> idf矩陣完成
>> tf-idf矩陣完成
>>>>>> 結束並存檔 (共1.69秒)
【2019-05-30】
>> 共有 278文章 / 3907詞 / 16365詞數
>> 次數Dataframe完成
>> tf矩陣完成
>> idf矩陣完成
>> tf-idf矩陣完成
>>>>>> 結束並存檔 (共84.15秒)
